# QUANTUM ESPRESSO Example App

**Author: Aliaksandr Yakutovich (THEOS, EPFL)**

This automatic workflow allows to optimize geometry and to compute the band structure of a material with a minimal set of input parameters.

It is powered by:
- [QUANTUM ESPRESSO](https://www.quantum-espresso.org/) as the quantum engine
- [AiiDA](http://www.aiida.net) as the automation platform
- [AiiDA-QUANTUMESPRESSO](https://github.com/aiidateam/aiida-quantumespresso) plugin
- Custom-made workflows for AiiDA to manage the selection of parameters, the error handling, ...
- [AppMode for Jupyter](http://github.com/oschuett/jupyter_appmode) to create a simple UI

### Example steps to run:
1. SCF
1. Relaxation
1. Band structure calculation

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
import sys
orig = sys.stderr
sys.stderr = sys.stdout

In [ ]:
import ipywidgets as ipw
from aiidalab_widgets_base import aiidalab_display, CodQueryWidget, CodeDropdown, StructureUploadWidget
from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.orm.data.base import Str
from aiida.work.run import run
from aiida.orm.utils import WorkflowFactory
from aiida.orm.data.array.kpoints import KpointsData
from aiida.orm.data.parameter import ParameterData
from aiida.orm import load_node

from time import sleep

In [ ]:
arrow_down = ipw.HTML("""<hr>
                         <br />
                         <center>
                         <i class="fa fa-arrow-down" style="color:#B0B0B0;font-size:12em;"></i>
                         </center>
                        """)
hr = ipw.HTML('<hr>')
run_btn = ipw.Button(description='Submit calculation')
code_group = CodeDropdown(input_plugin='quantumespresso.pw', text="Select code")
number_of_nodes = ipw.IntText(value=1,
                              step=1,description = "that will be run on",
                              disabled=False,
                              layout=ipw.Layout(width="180px"),
                              style={"description_width":"120px"},)

In [ ]:
structure_widget = StructureUploadWidget(examples=[('Silicon', 'structures/Si.xyz'), ('GaAs','structures/GaAs.xyz')],
                                         data_importers=[("COD", CodQueryWidget())],
                                         storable=False,
                                         node_class='StructureData')
def change_submit_calc_visibility(c):
    submit_out.layout.visibility = 'visible' if c['new'] else 'hidden'
structure_widget.observe(change_submit_calc_visibility, names=['has_structure'])

In [ ]:
def setup_calc():
    options =  {
        'max_wallclock_seconds': 3600*2,
        'resources':{
            'num_machines': number_of_nodes.value,
        }
    }
    kpoints = KpointsData()
    kpoints.set_kpoints_mesh([kpx.value, kpy.value, kpz.value])

    if code_group.selected_code is None:
        print ("Please select a code")
        return None

    parameters = {
        'SYSTEM': {
            'ecutwfc': 50.,
            'ecutrho': 200.,
        },
    }

    inputs = {
        'code': code_group.selected_code,
        'pseudo_family': Str(pseudo_family.value),
        'parameters': ParameterData(dict=parameters),
        'options': ParameterData(dict=options),
    }
    if 'optimized_structure' in globals():
        inputs['structure'] = optimized_structure
    else:
        inputs['structure'] = structure_widget.structure_node

    if run_type.label != 'bands':
        inputs['kpoints'] = kpoints
    return inputs

In [ ]:
def submit_calculation(b):
    global calculation
    inputs = setup_calc()
    if inputs is None:
        pass
    else:
        WorkChain = WorkflowFactory(run_type.value)
        calculation = run(WorkChain, **inputs)
        if 'output_structure' in calculation:
            optimized_structure = calculation['output_structure']
            global optimized_structure
        display(arrow_down)
        show_results()

In [ ]:
def calculation_settings():
    global run_type, kpx, kpy, kpz, pseudo_family
    run_type = ipw.ToggleButtons(
        options=[
            ('scf', 'quantumespresso.pw.base'),
            ('relax', 'quantumespresso.pw.relax'),
            ('bands', 'quantumespresso.pw.bands'),
        ],
        description='Calculation type:',
        style = {'description_width': 'initial'},
    )
    kpx = ipw.BoundedIntText(value=2, min=1, step=1, description='k-points', layout=ipw.Layout(width="144px"))
    kpy = ipw.BoundedIntText(value=2, min=1, step=1, description='', layout=ipw.Layout(width="50px"))
    kpz = ipw.BoundedIntText(value=2, min=1, step=1, description='', layout=ipw.Layout(width="50px"))
    kpoints = ipw.HBox([kpx, kpy, kpz])
    pseudo_family = ipw.ToggleButtons(
        options = {
            'SSSP efficiency': 'SSSP_efficiency_v1.0',
            'SSSP accuracy': 'SSSP_precision_v1.0',
        },
        description='Pseudopotential family:',
        style = {'description_width': 'initial'},
    )

    display(arrow_down,
            ipw.HBox([code_group, number_of_nodes, ipw.HTML("node(s)")]),
            run_type,
            pseudo_family,
            kpoints,
            run_btn)

In [ ]:
def show_results():
    for key in ['retrieved', 'output_structure', 'band_structure']:
        if key in calculation:
            aiidalab_display(calculation[key])
    calculation_settings()

In [ ]:
submit_out = ipw.Output(layout={'visibility':'hidden'})
with submit_out:
    run_btn.on_click(submit_calculation)
    calculation_settings()
display(structure_widget, submit_out)